In [1]:
# import a variety of packages
import requests
import yaml
from IPython.core.display import display, HTML
import numpy as np
import pandas as pd
!pip2 install --quiet pypathway # not instandard jupyter container so add manually
from pypathway import *
import pymongo

In [2]:
# Connect to the database
db = pymongo.MongoClient("mongo").MedBook

In [3]:
# Get the results of a specific job
job = db.jobs.find_one({"_id": "hzTfpy7kXzftFMJxe"})
up = pd.DataFrame(job["output"]["up_genes"])
down = pd.DataFrame(job["output"]["down_genes"])
top5percent = pd.DataFrame(job["output"]["top5percent_genes"])
print "Outlier analysis for", job["args"]["sample_label"],"against", job["args"]["sample_group_name"]

Outlier analysis for ckcc/TH03_0118_S01_RNASeq against Compendium 2 final samples 2016/10/24


In [4]:
diagram = PublicDatabase.search_kegg("mapk", organism="hsa")[0].load()

In [5]:
up_ids = [e.id for e in diagram.entities if e.display_name in set(up["gene_label"].values)]
down_ids = [e.id for e in diagram.entities if e.display_name in set(down["gene_label"].values)]
top5percent_ids = [e.id for e in diagram.entities if e.id not in up_ids and e.id not in down_ids and e.display_name in set(top5percent["gene_label"].values)]

In [6]:
# Use colorblind safe colors: http://mkweb.bcgsc.ca/colorblind/
up_color = Option(default=[Prop(bg_color="rgb(213,94,0)", color="white")])
down_color = Option(default=[Prop(bg_color="rgb(0,114,178)", color="white")])
top5percent_color = Option(default=[Prop(bg_color="rgb(230,159,0)", color="white")])
opaque = Option(default=[Prop(bg_color="white", color="white", opacity=0.8)])

diagram.integrate([e.id for e in diagram.entities],
                  [up_color if i in up_ids
                   else down_color if i in down_ids 
                   else top5percent_color if i in top5percent_ids 
                   else opaque for i in [e.id for e in diagram.entities]])

diagram.draw()